## Import libraries

In [1]:
import os
import sys

import pandas as pd
import geopandas as gpd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns

from pandas.api.types import CategoricalDtype

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/home/jovyan/accesibilidad-urbana/aup/data.py:25: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(


In [2]:
city = 'CDMX'

## Load hexs with vegetation in alcaldías of interest

### NDVI Load data - for city and res

NDVI's hex were processed using the correct urban+rural hexs configuration on Script 21.

In [3]:
# Downloading NDVI res 10 data (aprox. 150-200s)
ndvi_schema = 'raster_analysis'
ndvi_table = 'ndvi_analysis_hex'
res = 10
query = f"SELECT * FROM {ndvi_schema}.{ndvi_table} WHERE \"city\" LIKE \'{city}\' AND \"res\" = {res}"
ndvi_res10 = aup.gdf_from_query(query, geometry_col='geometry')

# Shorten data
ndvi_res10 = ndvi_res10[['hex_id','ndvi_median']]

# Show
print(ndvi_res10.shape)
ndvi_res10.head(2)

(59557, 2)


,hex_id,ndvi_median
0,8a4995baa487fff,0.139817
1,8a4995b8cb57fff,0.108460


### NDVI Data treatment - Create vegetation categories

In [4]:
# Create vegetation categories
ndvi_res10['ndvi_median_rango'] = np.nan
ndvi_res10.loc[ndvi_res10.ndvi_median>=0.6 , 'ndvi_median_rango'] = 'Alta densidad vegetal'
ndvi_res10.loc[(ndvi_res10.ndvi_median>=0.4 )&
                (ndvi_res10.ndvi_median<0.6), 'ndvi_median_rango'] = 'Moderada densidad vegetal'
ndvi_res10.loc[(ndvi_res10.ndvi_median>=0.2)&
                (ndvi_res10.ndvi_median<0.4), 'ndvi_median_rango'] = 'Mínima densidad vegetal'
ndvi_res10.loc[(ndvi_res10.ndvi_median>=0.1)&
                (ndvi_res10.ndvi_median<0.2), 'ndvi_median_rango'] = 'Suelo'
ndvi_res10.loc[(ndvi_res10.ndvi_median<0.1), 'ndvi_median_rango'] = 'Suelo artificial/Agua/Piedra'

# Ordering data
categories = ['Suelo artificial/Agua/Piedra', 'Suelo', 'Mínima densidad vegetal', 'Moderada densidad vegetal', 'Alta densidad vegetal']
ndvi_res10['ndvi_median_rango'] = pd.Categorical(ndvi_res10['ndvi_median_rango'], categories=categories, ordered=True)

# Show
print(ndvi_res10.shape)
ndvi_res10.head(2)

(59557, 3)


,hex_id,ndvi_median,ndvi_median_rango
0,8a4995baa487fff,0.139817,Suelo
1,8a4995b8cb57fff,0.108460,Suelo


### NDVI Data treatment - Filter for places with vegetation

In [5]:
#Keeping High-moderate-minimal vegetation density
data_withveg = ndvi_res10.loc[(ndvi_res10.ndvi_median_rango == 'Alta densidad vegetal') |
               (ndvi_res10.ndvi_median_rango == 'Moderada densidad vegetal')|
               (ndvi_res10.ndvi_median_rango == 'Mínima densidad vegetal')].copy()

# Show
print(data_withveg.shape)
data_withveg.head(2)

(35317, 3)


,hex_id,ndvi_median,ndvi_median_rango
8,8a4995b8d277fff,0.283320,Mínima densidad vegetal
11,8a4995b81107fff,0.215763,Mínima densidad vegetal


### hex_gdf Load data - (Contiene alcaldías)

In [7]:
# Download hex_gdf  (aprox. 150-200s)
hex_schema = 'hexgrid'
hex_table = 'hexgrid_10_city_2020'
query = f"SELECT * FROM {hex_schema}.{hex_table} WHERE \"city\" LIKE \'{city}\'"
hex_gdf = aup.gdf_from_query(query, geometry_col='geometry')

# Show
print(hex_gdf.shape)
hex_gdf.head(2)

(86349, 6)


,hex_id_10,geometry,CVEGEO,NOMGEO,city,type
0,8a4995b8cb57fff,"POLYGON ((-99.19358 19.48193, -99.19293 19.482...",09002,Azcapotzalco,CDMX,urban
1,8a4995b83a4ffff,"POLYGON ((-99.15235 19.48171, -99.15170 19.482...",09002,Azcapotzalco,CDMX,urban


### hex_gdf Data treatment - Filter for alcaldías of interest

In [8]:
# Filter hex_gdf for municipalities where Cutzamala system provides water
alcaldias_cutzamala = ['Álvaro Obregón','Azcapotzalco', 'Cuajimalpa de Morelos','Iztacalco', 
                       'Iztapalapa','La Magdalena Contreras','Tlalpan', 'Benito Juárez', 
                       'Cuauhtémoc','Miguel Hidalgo', 'Venustiano Carranza']
hex_gdf_f = hex_gdf.loc[hex_gdf.NOMGEO.isin(alcaldias_cutzamala)]

# Shorten data
hex_gdf_f = hex_gdf_f[['hex_id_10','NOMGEO','geometry']]
hex_gdf_f.rename(columns={'hex_id_10':'hex_id'},inplace=True)

# Show
print(hex_gdf_f.shape)
hex_gdf_f.head(2)

(49335, 3)


,hex_id,NOMGEO,geometry
0,8a4995b8cb57fff,Azcapotzalco,"POLYGON ((-99.19358 19.48193, -99.19293 19.482..."
1,8a4995b83a4ffff,Azcapotzalco,"POLYGON ((-99.15235 19.48171, -99.15170 19.482..."


### Data with veg (NDVI) + alcaldías (hex_gdf)

In [9]:
# Inner join to keep data with veg of alcaldías of interest only
data_withveg_alcaldias = data_withveg.merge(hex_gdf_f,on='hex_id',how='inner')

# Show
print(data_withveg_alcaldias.shape)
data_withveg_alcaldias.head(2)

(18981, 5)


,hex_id,ndvi_median,ndvi_median_rango,NOMGEO,geometry
0,8a4995b8d277fff,0.283320,Mínima densidad vegetal,Azcapotzalco,"POLYGON ((-99.20948 19.50623, -99.20883 19.506..."
1,8a4995b81107fff,0.215763,Mínima densidad vegetal,Azcapotzalco,"POLYGON ((-99.16277 19.46651, -99.16212 19.466..."


## NDMI Load data - Load using city, res and tupple of months of analysis

In [14]:
# Hexs of interest (with vegetation and in alcaldias of interest)

# Complete dataset NDMI res 10 data 
complete_ndmi_schema = 'raster_analysis'
complete_ndmi_table = 'ndmi_complete_dataset_hex'
res = 10

# OPTION A - Downloading complete dataset and then filtering for months of interest (aprox. 700-1000s)
#query = f"SELECT * FROM {complete_ndmi_schema}.{complete_ndmi_table} WHERE \"city\" LIKE \'{city}\' AND \"res\"={res}"
#complete_ndmi_res10 = aup.df_from_query(query)
#months_ofinterest = [1,2,3,4,5]
#complete_ndmi_res10_f = complete_ndmi_res10.loc[complete_ndmi_res10.month.isin(months_ofinterest)]

# OPTION B - Using hex_lsts and quering with tupple (First time 3494s, second time 3409s)
#hexs_lst = list(data_withveg_alcaldias.hex_id.unique())
#query = f"SELECT * FROM {complete_ndmi_schema}.{complete_ndmi_table} WHERE \"city\" LIKE \'{city}\' AND \"res\"={res} AND \"hex_id\" IN {str(tuple(hexs_lst))}"
#complete_ndmi_res10 = aup.df_from_query(query)

# OPTION C - Downloading complete dataset filtering for months of interest in query (806s)
months_ofinterest = [1,2,3,4,5]
query = f"SELECT * FROM {complete_ndmi_schema}.{complete_ndmi_table} WHERE \"city\" LIKE \'{city}\' AND \"res\"={res} AND \"month\" IN {str(tuple(months_ofinterest))}"
complete_ndmi_res10 = aup.df_from_query(query)

# Shorten data
complete_ndmi_res10_f = complete_ndmi_res10[['hex_id','ndmi','month','year']]

# Show
print(complete_ndmi_res10_f.shape)
complete_ndmi_res10_f.head(2)

(1911910, 4)


,hex_id,ndmi,month,year
0,8a4995b8cb57fff,-0.048705,1,2023
1,8a4995b8dd07fff,-0.032961,1,2023


## Data treatment

### Data treatment - Merge ndmi for places with vegetation

In [17]:
# Inner merge to keep ndmi data for hexs with vegetation only
ndvi_ndmi_gdf = data_withveg.merge(complete_ndmi_res10_f, on='hex_id',how='inner')

# Show
print(ndvi_ndmi_gdf.shape)
ndvi_ndmi_gdf.head(2)

(1063010, 6)


,hex_id,ndvi_median,ndvi_median_rango,ndmi,month,year
0,8a4995b8d277fff,0.28332,Mínima densidad vegetal,0.025837,1,2023
1,8a4995b8d277fff,0.28332,Mínima densidad vegetal,0.006358,2,2023


## Data analysis

### Método elegido - Group by + transform data

In [37]:
# Isolate categories and data
data_analysis = ndvi_ndmi_gdf[['hex_id','year','ndmi']].copy()

# Group by
grouped_by = data_analysis.groupby(['hex_id','year']).mean()
grouped_by.reset_index(inplace=True)

# Transform data
# GDF with unique rows for each geometry (hex)
mean_ndmi_bydate = hex_gdf_f.copy()

# Loop that transforms data to columns
for year in grouped_by.year.unique():
    
    # Isolate new column data (In this case, year)
    idx = grouped_by.year == year
    df_tmp = grouped_by.loc[idx,['hex_id','ndmi']].copy()
    
    # Assign as column name (In this case, year)
    df_tmp = df_tmp.rename(columns={'ndmi':f"ene-may_{year}"})
    
    # Merge to gdf with unique geometry IDs (In this case, hex_id)
    mean_ndmi_bydate = mean_ndmi_bydate.merge(df_tmp, on='hex_id',how='inner')

mean_ndmi_bydate

,hex_id,NOMGEO,geometry,ene-may_2018,ene-may_2019,ene-may_2020,ene-may_2021,ene-may_2022,ene-may_2023
0,8a4995b81b27fff,Azcapotzalco,"POLYGON ((-99.17277 19.46465, -99.17213 19.465...",0.044359,0.042229,0.061561,0.056257,0.039622,0.024283
1,8a4995baa49ffff,Azcapotzalco,"POLYGON ((-99.17202 19.46359, -99.17138 19.464...",0.036326,0.035393,0.055935,0.050881,0.037117,0.021332
2,8a4995baa6f7fff,Azcapotzalco,"POLYGON ((-99.17996 19.47076, -99.17931 19.471...",0.063569,0.064508,0.089804,0.075977,0.051897,0.041203
3,8a4995b8da0ffff,Azcapotzalco,"POLYGON ((-99.21230 19.49826, -99.21165 19.498...",0.047337,0.068382,0.027007,-0.003597,0.041621,-0.005511
4,8a4995bab7affff,Azcapotzalco,"POLYGON ((-99.19955 19.48036, -99.19890 19.480...",0.018522,0.010196,0.018774,0.018388,0.017814,0.008704
...,...,...,...,...,...,...,...,...,...
18976,8a4995b8640ffff,Venustiano Carranza,"POLYGON ((-99.11389 19.44359, -99.11324 19.444...",0.028383,0.024993,0.037698,0.039310,0.035020,0.019361
18977,8a4995bb3d57fff,Venustiano Carranza,"POLYGON ((-99.09936 19.41918, -99.09872 19.419...",0.054442,0.058805,0.079736,0.076332,0.059195,0.041614
18978,8a4995bb1597fff,Venustiano Carranza,"POLYGON ((-99.11205 19.41712, -99.11140 19.417...",0.021455,0.009891,0.021921,0.022106,0.021361,0.007051
18979,8a4995bb070ffff,Venustiano Carranza,"POLYGON ((-99.10278 19.41005, -99.10213 19.410...",0.013737,0.005663,0.012716,0.019606,0.014213,0.011818


### Método 1 - 1425s

In [18]:
metodo_1 = False

In [19]:
if metodo_1:
    mean_ndmi_bydate = hex_gdf_f.copy()
    
    i = 0
    for year in data_withveg_alcaldias.year.unique():
        for month in data_withveg_alcaldias.month.unique():
            for alcaldia in data_withveg_alcaldias.NOMGEO.unique():
    
                print(f"Calculating mean ndmi in {alcaldia} date {month}/{year}.")
                    
                data_df = data_withveg_alcaldias.loc[(data_withveg_alcaldias.year == year) &
                                                     (data_withveg_alcaldias.month == month) &
                                                     (data_withveg_alcaldias.NOMGEO == alcaldia)].copy()
    
                for hex_id in data_df['hex_id'].unique():
                    # Get each mean hex_id data for current year, month and alcaldía
                    hex_data = data_df.loc[data_df['hex_id'] == hex_id].copy()
                    mean_ndmi = hex_data['ndmi'].mean()
                    # Register data
                    idx = mean_ndmi_bydate['hex_id'] == hex_id
                    mean_ndmi_bydate.loc[idx,f"{year}_{month}"] = mean_ndmi
    
    # Show
    print(mean_ndmi_bydate.shape)
    mean_ndmi_bydate.head(2)

### Método 2 - Transform data to columns and calculate mean on row axis (Canceled)

In [20]:
metodo_2 = False

In [21]:
if metodo_2:
    data_analysis = data_withveg_alcaldias.copy()
    # Create date column
    data_analysis['year'] = data_analysis['year'].astype('str')
    data_analysis['month'] = data_analysis['month'].astype('str') 
    data_analysis['date'] =  data_analysis['year'] + "_"+ data_analysis['month']
    # Drop columns no longer in use
    data_analysis.drop(columns=['ndvi_median','ndvi_median_rango','month','year'],inplace=True)
    
    # Show
    print(data_analysis.shape)
    data_analysis.head(2)

In [22]:
if metodo_2:
    # TRANSFORM DATA FROM MULTIPLE ROWS TO COLUMNS
    ######################################################################### CÓDIGO BASE PARA REPOSICIONAR DATOS (Script 15-15min-cities)
    # delete duplicates and keep only one point for each node
    #nodes_geom = nodes.drop_duplicates(subset='osmid', keep="last")[['osmid','geometry','metropolis']].copy()
    #nodes_analysis = nodes_geom.copy()
    
    # relate time data to each point
    #for amenidad in list(nodes.amenity.unique()):
    
    #    nodes_tmp = nodes.loc[nodes.amenity == amenidad,['osmid','time']]
    #    nodes_tmp = nodes_tmp.rename(columns={'time':amenidad})
    
    #    if nodes_tmp[amenidad].mean() == 0:
    #        nodes_tmp[amenidad] = np.nan
    
    #    nodes_analysis = nodes_analysis.merge(nodes_tmp, on='osmid')
    
    ######################################################################### PARA ESTE CÓDIGO
    #nodes_analysis (gdf con 1 row para cada geometría) --> proviene de data_analysis 
    #amenidad --> date (lo que serán las columnas en el nuevo gdf)
    #osmid --> hex_id (columna de ID de las geometrías)
    #time --> ndmi (columna con datos a reposicionar)
    
    ######################################################################### APLICACIÓN
    # GDF with unique rows for each geometry
    mean_ndmi_bydate = data_analysis.drop_duplicates(subset='hex_id', keep='last')[['hex_id','geometry','NOMGEO']].copy()
    
    # Loop that transforms data
    for date in data_analysis.date.unique():
        
        # Isolate new column data (In this case, date)
        idx = data_analysis.date == date
        df_tmp = data_analysis.loc[idx,['hex_id','ndmi']].copy()
        
        # Assign as column name (In this case, date)
        df_tmp = df_tmp.rename(columns={'ndmi':date})
        
        # Merge to gdf with unique geometry IDs (In this case, hex_id)
        mean_ndmi_bydate = mean_ndmi_bydate.merge(df_tmp, on='hex_id',how='inner')
    
    mean_ndmi_bydate

## Save

In [40]:
save_dir = f'../../../data/external/temporal_fromjupyter/'
mean_ndmi_bydate.to_file(save_dir+"cutzamala/cdmx_enemay_ndmi.gpkg")